In [27]:
%run import_packages/import_packages.ipynb

%run function/functions.ipynb

In [28]:
applications,jobs,test_users,users,users_history = import_csv()

In [29]:
df_job = create_df_job(jobs,applications,users_history,users)
df_job_unique = df_unique_job_id(users_history,jobs)
df_job = df_job.merge(df_job_unique,on="JobTitle")

df_job

,UserID,JobTitle,NB,jobID
0,23,assisting customers,1,1
1,23,manager,1,0
2,24848,manager,1,0
3,115398,manager,1,0
4,132309,manager,1,0
...,...,...,...,...
17880,1470222,project manager/program manager – application ...,1,6978
17881,1470222,"project, product manager/team lead/ sr. system...",1,6980
17882,1470222,senior manager of it corporate systems,1,7484
17883,1470245,consultant/temporary,3,6982


In [30]:
df_test = df_create_test(test_users,users_history,df_job_unique)
df_test

,UserID,JobTitle,NB,jobID
0,12924,customer service/ sales representative,1,50
1,12924,homemaker,2,51
2,237967,homemaker,1,51
3,12924,substitute teacher,1,49
4,140031,substitute teacher,1,49
...,...,...,...,...
1001,1466313,"vice president, database marketing",1,6929
1002,1466313,"vice president, marketing consulting",1,6926
1003,1471500,cocktail server,2,6984
1004,1471500,ics associate /overnight stock/truck unloader,1,6985


In [31]:
input_data = pd.concat([df_test,df_job])
input_data.reset_index(drop=True,inplace=True)

In [56]:
size=len(df_test)
test_jobid = input_data.iloc[:size].jobID
test_jobid = test_jobid.unique().tolist()

test_UserID = input_data.iloc[:size].UserID
test_UserID = test_UserID.unique().tolist()

In [33]:
len(test_jobid)

824

In [34]:
sparse_user_item = csr_matrix((input_data['NB'].astype(float), 
                               (input_data['UserID'], 
                                input_data['jobID'])))

In [35]:
sparse_user_item

<1472023x9960 sparse matrix of type '<class 'numpy.float64'>'
	with 18891 stored elements in Compressed Sparse Row format>

In [36]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50)

# train the model on a sparse matrix of user/item/confidence weights
model.fit(sparse_user_item)

In [12]:
predict_job = [model.similar_items(i,N=4)[0].tolist()[1:] for i in test_jobid]
predict_similarity = [model.similar_items(i,N=4)[1].tolist()[1:] for i in test_jobid]

In [13]:
result=pd.DataFrame({"jobID":test_jobid,"predict_job":predict_job,"predict_similarity":predict_similarity})

predict_job_title=[df_job_unique.iloc[predict].JobTitle.tolist() for predict in result.predict_job]

result["predict_job_title"]=predict_job_title

In [14]:
result = df_test.merge(result,on="jobID")

In [15]:
result

,UserID,JobTitle,NB,jobID,predict_job,predict_similarity,predict_job_title
0,12924,customer service/ sales representative,1,50,"[51, 6675, 6679]","[0.9104598760604858, 0.9102471470832825, 0.909...","[homemaker, barkey associate, patrol officer/ ..."
1,12924,homemaker,2,51,"[50, 175, 2932]","[0.9104598760604858, 0.9095978736877441, 0.909...","[customer service/ sales representative, stude..."
2,237967,homemaker,1,51,"[50, 175, 2932]","[0.9104598760604858, 0.9095978736877441, 0.909...","[customer service/ sales representative, stude..."
3,12924,substitute teacher,1,49,"[5131, 5132, 750]","[0.9843307733535767, 0.9843292236328125, 0.979...","[teacher's aid/ substitute, painter/prop manag..."
4,140031,substitute teacher,1,49,"[5131, 5132, 750]","[0.9843307733535767, 0.9843292236328125, 0.979...","[teacher's aid/ substitute, painter/prop manag..."
...,...,...,...,...,...,...,...
1001,1466313,"vice president, database marketing",1,6929,"[6925, 6930, 6146]","[0.9974234700202942, 0.9911640286445618, 0.982...",[director of marketing and communication servi...
1002,1466313,"vice president, marketing consulting",1,6926,"[6928, 6924, 2898]","[0.9940532445907593, 0.9856798052787781, 0.983...","[director, marketing and sales, director of ma..."
1003,1471500,cocktail server,2,6984,"[6985, 3024, 3022]","[0.9999999403953552, 0.9998452067375183, 0.999...",[ics associate /overnight stock/truck unloader...
1004,1471500,ics associate /overnight stock/truck unloader,1,6985,"[6984, 3022, 3024]","[0.9999999403953552, 0.9998453855514526, 0.999...","[cocktail server, bar manager/bartender, waitr..."


# Reco

In [110]:
recommend_job = [model.recommend(id, sparse_user_item[id])[0][:3].tolist() for id in test_UserID]
recommend_job_sim = [model.recommend(id, sparse_user_item[id])[1][:3].tolist() for id in test_UserID]

In [116]:
userid=1303999
model.recommend(userid, sparse_user_item[userid])

(array([ 171,  214,   92,  512,   98,  912, 1240, 1239,  236,  278],
       dtype=int32),
 array([0.11328357, 0.0999704 , 0.07472458, 0.06744396, 0.06386355,
        0.05273058, 0.0522202 , 0.04531711, 0.04311183, 0.04043042],
       dtype=float32))

In [111]:
test = pd.DataFrame(test_UserID,columns=["UserID"])
test = test.merge(df_history_list(users_history),on="UserID")
test["reco_job"] = recommend_job
test["reco_job_sim"]=recommend_job_sim

reco_job_title=[df_job_unique.iloc[predict].JobTitle.tolist() for predict in test.reco_job]
test["reco_job_title"]=reco_job_title

In [114]:
test

,UserID,users_history,reco_job,reco_job_sim,reco_job_title
0,12924,"[substitute teacher, customer service/ sales r...","[214, 219, 838]","[0.051453713327646255, 0.04104168340563774, 0....","[security officer, program coordinator, case m..."
1,237967,"[security officer, security officer/dispatcher...","[278, 858, 125]","[0.2928774952888489, 0.19467642903327942, 0.15...","[security guard, truck driver, medical assistant]"
2,140031,"[editor, freelance writer/editor, substitute t...","[171, 437, 512]","[0.02974502369761467, 0.02588551491498947, 0.0...","[office manager, server, account manager]"
3,356194,"[administrative assistant, substitute teacher,...","[1738, 7755, 9932]","[0.04752327874302864, 0.04097697138786316, 0.0...","[executive assistant, receptionist/ administra..."
4,1303999,"[dean of students, director - presidential off...","[171, 214, 92]","[0.11328356713056564, 0.09997040033340454, 0.0...","[office manager, security officer, sales manager]"
...,...,...,...,...,...
258,1373611,"[business owner, casework manager, supervisor ...","[1738, 1469, 986]","[0.008472699671983719, 0.0020401719957590103, ...","[executive assistant, pharmacy technician, fin..."
259,1404139,"[inventory control specialist, senior inventor...","[16, 92, 966]","[2.0521646604265698e-07, 1.9177510068857373e-0...","[operations manager, sales manager, supervisor]"
260,1438650,"[bakery chef, airmen first class]","[966, 9773, 1469]","[5.065645041213429e-07, 5.0364411663395e-07, 4...","[supervisor, order fulfillment associate, phar..."
261,1454754,"[senior grant accountant, senior accountant (t...","[62, 281, 283]","[0.03163096308708191, 0.01489391177892685, 0.0...","[staff accountant, accountant, senior accountant]"
